## Appendix: Save Results

In [ ]:
# Save portfolios to CSV
portfolios = pd.DataFrame({
    'Markowitz': weights_markowitz,
    'Genetic_Algorithm': weights_ga,
}, index=tickers)

# Add QAOA (with NaN for missing assets)
qaoa_full = pd.Series(0.0, index=tickers)
qaoa_full[subset_tickers] = weights_qaoa
portfolios['QAOA'] = qaoa_full

portfolios.to_csv('../results/portfolio_comparison.csv')
print("\nResults saved to: results/portfolio_comparison.csv")
print("\nPortfolio Weights:")
print(portfolios[portfolios > 0.001])